In [2]:
import pickle
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.util import ngrams
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

In [3]:
with open("keywords.pkl", "rb") as f:
    keywords = pickle.load(f)        

In [4]:
print(keywords[3])

['water qualiti quantiti', 'water', 'waterbodi', 'water bodi', 'watercours', 'evapor', 'transpir', 'surfac water', 'ground water', 'runoff', 'contamin', 'contamin', 'water use', 'hydrolog', 'hydrolog', 'hydro', 'hydrostat', 'salin', 'blast', 'withdraw', 'flow', 'peak', 'basin', 'inter-basin', 'water qualiti', 'water qualiti test', 'water tabl', 'contain', 'sediment', 'sewer', 'wast', 'wastewat', 'biosolid', 'sludg', 'septag', 'groundwat', 'groundwater-rel', 'aquif', 'streamflow', 'aquat', 'acquat', 'river', 'hydrometr', 'watersh', 'waterfal', 'sea', 'ocean', 'lake', 'pond', 'fjord', 'wadi', 'run', 'reservoir', 'lagoon', 'bay', 'harbor', 'well', 'well-wat', 'surface-wat', 'surfacewat', 'hot spring', 'creek', 'tidal', 'subtid', 'mercuri', 'contamin']


In [5]:
labels_list = ['Physical and meteorological environment',
                'Soil and soil productivity',
                'Vegetation',
                'Water quality and quantity',
                'Fish and fish habitat',
                'Wetlands',
                'Wildlife and wildlife habitat',
                'Species at Risk or Species of Special Status and related habitat',
                'Greenhouse gas (GHG) emissions and climate change',
                'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
                'Air emissions',
                'Acoustic environment',
                'Electromagnetism and Corona Discharge',
                'Human occupancy and resource use',
                'Heritage resources',
                'Navigation and navigation safety',
                'Traditional land and resource use',
                'Social and cultural well-being',
                'Human health and aesthetics',
                'Infrastructure and services',
                'Employment and economy',
                'Environmental Obligations',
                'Rights of Indigenous Peoples']

In [6]:
# df = pd.read_csv('./data/ESA_website_ENG_with_VCs_onehot.csv')
df = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv', index_col=0)

In [7]:
df.drop(columns=['label'], inplace=True)

In [8]:
df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...


In [197]:
table_texts = df['text'].tolist()

tokenized_table_texts = []
stemmer = PorterStemmer()

for i, table_text in enumerate(table_texts):
    processed_text = word_tokenize(table_text)
    processed_text = [stemmer.stem(w) for w in processed_text if w not in stopwords.words("english")]
    table_ngram_list = []
    for n in range(1, 7):
        table_ngrams = list(ngrams(processed_text, n))
        table_ngram_list.extend([" ".join(table_gram) for table_gram in table_ngrams])

    tokenized_table_texts.append(table_ngram_list)
    
print(tokenized_table_texts[0][0:200])

['tabl', '3', 'summari', 'OF', 'aquat', 'field', 'work', 'and', 'aborigin', 'field', 'studi', 'particip', 'for', 'the', 'project', 'survey', 'date', 'aborigin', 'commun', 'detail', 'juli', '9', '12', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'summer', '2011', 'open', 'water', 'aquat', 'assess', 'septemb', '18', 'octob', '10', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'fall', '2011', 'open', 'water', 'aquat', 'assess', 'februari', '26', 'march', '12', '2012', 'blueberri', 'river', 'first', 'nation', 'doig', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'kelli', 'lake', 'cree', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'winter', '2012', 'aquat', 'habitat', 'investig', 'tabl 3', '3 summari', 's

In [199]:
with open('tokenized_table_texts.pkl', 'wb') as f:
    pickle.dump(tokenized_table_texts, f)

In [9]:
with open('tokenized_table_texts.pkl', 'rb') as f:
    tokenized_table_text = pickle.load(f)

In [12]:
print(tokenized_table_text[0][:10], '\n', tokenized_table_text[1][:10])

['tabl', '3', 'summari', 'OF', 'aquat', 'field', 'work', 'and', 'aborigin', 'field'] 
 ['tabl', '4', 'summari', 'OF', 'watercours', 'cross', 'along', 'the', 'north', 'montney']


In [10]:
for label in labels_list:
    df[f'{label}'] = 0
    # df[f'{label} - Number of Matches'] = 0
    # df[f'{label} Relevance'] = 0

df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
def label_with_keywords(i, table_text, keywords_for_label, label):
    number_of_matches = len([word for word in keywords_for_label if word in table_text])
    df[f'{label}'][i] = number_of_matches

In [14]:
def string_total_sum_match_2_lists(list_1, list_2):
    """
    This function takes two lists of strings and counts the number of total matches between the two lists, including duplicates.
    The function returns the number of total matches.
    """
    # Initialize the count variable.
    count = 0
    # Iterate over the first list.
    for item in list_1:
        # If the item is in the second list, increment the count variable.
        if item in list_2:
            count += 1
    # Return the count variable.
    return count

In [15]:
df.text[0]

'TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT Survey Date Aboriginal Communities Detail July 9 to 12 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Summer 2011 open water aquatic assessments September 18 to October 10 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Fall 2011 open water aquatic assessments February 26 to March 12 2012 Blueberry River First Nation Doig River First Nation Halfway River First Nation Kelly Lake Cree First Nation McLeod Lake Indian Band North East Mtis Association Winter 2012 aquatic habitat investigations '

In [16]:
for i, table_text in enumerate(tokenized_table_text):
    for keywords_for_label, label in zip(keywords, labels_list):
        number_of_matches = string_total_sum_match_2_lists(table_text, keywords_for_label)
        df[f'{label}'][i] = number_of_matches
        
with open("esa_index_ENG_labeled.pkl", "wb") as f:
    pickle.dump(df, f)

df.head()

<ipython-input-16-94bef19dd2c9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = number_of_matches


In [ ]:
df.to_csv('./mydata/esa_index_keyword_labeled_2aug2021.csv')

In [ ]:
with open("esa_index_ENG_labeled.pkl", "rb") as f:
    df = pickle.load(f)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, validation_df = train_test_split(df, test_size=300, random_state=42)
validation_df_labels = validation_df.iloc[:, 33:]
validation_df_labels.head(2)

,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
8889,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEA...,2,1,1,3,3,1,2,0,0,10,2,2,0,4,2,1,2,1,0,11,9,0,4
27753,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guide...,0,3,39,0,0,0,0,13,2,0,0,2,0,1,0,6,0,1,0,6,4,0,1


In [ ]:
validation_df_labels['Comments'] = 'N/A'

In [ ]:
train_df.to_csv("./data/esa_keyword_labeled_train_df_2aug2021.csv", index=False)
validation_df.to_csv("./data/esa_keyword_labeled_validation_2aug2021df.csv", index=False)
validation_df_labels.to_csv("./data/esa_keyword_labeled_validation_df_labels_2aug2021.csv", index=False)

In [44]:
labeled_df = pd.read_csv('./data/project-1-at-2021-07-09-00-52-898dbd22.csv')

In [45]:
labeled_df

,Unnamed: 0,text,label,id,sentiment,annotator,annotation_id
0,8889,TABLE 8A.2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEAR CANYON SECTION Map ID s 1 s Title Proponent Description Construction Schedule s 2 s Assessment Type Quantitative or Qualitative Physical Environment and Soils LSA Water Quality and Quantity RSA Air Quality RSA Acoustic Environment RSA Fish and Fish Habitat RSA Wetlands RSA Vegetation RSA Wildlife and Wildlife Habitat RSA HORU RSA Heritage Resources RSA TRLU RSA Community RSA 13 30 79 12 W6M to...,-1,1,"{""choices"": [""Physical and meteorological environment"", ""Soil and soil productivity"", ""Vegetation"", ""Water quality and quantity"", ""Fish and fish habitat"", ""Wetlands"", ""Wildlife and wildlife habitat"", ""Acoustic environment""]}",jacques.thibodeau@cer-rec.gc.ca,1


In [91]:
for i in range(len(labeled_df)):
    chosen_labels = ast.literal_eval(labeled_df.sentiment[i])["choices"]
    for label in labels_list:
        if label in chosen_labels:
            df[f'{label}'][i] = 1

<ipython-input-91-cb7868f289e9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = 1


In [92]:
df.iloc[:,36:58]

,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations
0,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0
1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1
2,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1
3,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1
4,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28886,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,1
28887,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1
28888,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1
28889,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1


In [ ]:
for i in range(len(labeled_df)):
    

In [117]:
str('"choices" ' + str(df.iloc[0, 36:][df.iloc[0, 36:] == 1].index.values.tolist()) + '}').replace('\'', '')

'"choices" [Physical and meteorological environment, Soil and soil productivity, Vegetation, Water quality and quantity, Fish and fish habitat, Wetlands, Wildlife and wildlife habitat, Acoustic environment, Navigation and navigation safety, Traditional land and resource use, Social and cultural well-being, Rights of Indigenous Peoples]}'

In [119]:
type({"choices": ["Physical and meteorological environment", "Soil and oil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Acoustic environment"]})

dict

In [138]:
myDict = {}
myDict["choices"] = df.iloc[0, 36:][df.iloc[0, 36:] == 1].index.values.tolist()

In [139]:
myDict

{'choices': ['Physical and meteorological environment',
  'Soil and soil productivity',
  'Vegetation',
  'Water quality and quantity',
  'Fish and fish habitat',
  'Wetlands',
  'Wildlife and wildlife habitat',
  'Acoustic environment',
  'Navigation and navigation safety',
  'Traditional land and resource use',
  'Social and cultural well-being',
  'Rights of Indigenous Peoples']}

In [141]:
import json

json.dumps(myDict)

'{"choices": ["Physical and meteorological environment", "Soil and soil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Acoustic environment", "Navigation and navigation safety", "Traditional land and resource use", "Social and cultural well-being", "Rights of Indigenous Peoples"]}'

In [247]:
import json

with open('./mydata/project-1-at-2021-07-17-04-15-91c3a72e.json', 'r') as f:
    diction = json.load(f)


In [266]:
pd.DataFrame(diction[13:16])

,id,annotations,predictions,file_upload,data,meta,created_at,updated_at,project
0,5,"[{'id': 5, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 9544, 'text': 'Table 5.21-8: Major Proposed, Approved, and Recently Completed Pro...",{},2021-06-08T21:27:06.468686Z,2021-06-08T21:27:06.468698Z,1
1,4,"[{'id': 4, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 22455, 'text': 'TABLE 3.2 CONSULTATION ACTIVITIES WITH PROVINCIAL AGENCIES Stake...",{},2021-06-08T21:27:06.468603Z,2021-06-08T21:27:06.468611Z,1
2,3,"[{'id': 3, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 4165, 'text': 'TABLE D4 HOPE TO BURNABY OBSERVED PLANT SPECIES ï¿½ BY TYPE AND CO...",{},2021-06-08T21:27:06.468566Z,2021-06-08T21:27:06.468574Z,1


In [259]:
import pandas as pd

testing_df = pd.DataFrame(diction)
# testing_df.annotations[0]
with open('./mydata/project-1-at-2021-07-17-04-15-91c3a72e.json', 'r') as f:
    testing_df = pd.read_json(f)

testing_df

,id,annotations,predictions,file_upload,data,meta,created_at,updated_at,project
0,1,"[{'id': 1, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 8889, 'text': 'TABLE 8A.2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN TH...",{},2021-06-08 21:27:06.468460+00:00,2021-07-10 21:55:17.524780+00:00,1
1,17,"[{'id': 17, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 15949, 'text': 'Table B-1 Progress Farrell Creek West Gas Plant B-088-I/094-B-01 ...",{},2021-06-08 21:27:06.469107+00:00,2021-06-08 21:27:06.469114+00:00,1
2,16,"[{'id': 16, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 15105, 'text': 'Table 8-19 Exposure Point Concentrations in Marine Organisms Expo...",{},2021-06-08 21:27:06.469073+00:00,2021-06-08 21:27:06.469080+00:00,1
3,15,"[{'id': 15, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 10843, 'text': 'Table 6-3 Recommended Mitigation Measures for Fish and Fish Habit...",{},2021-06-08 21:27:06.469041+00:00,2021-06-08 21:27:06.469048+00:00,1
4,14,"[{'id': 14, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 10810, 'text': 'Table 2-6 Residual Effects on Land Resource Use Project Phase Mi...",{},2021-06-08 21:27:06.469005+00:00,2021-06-08 21:27:06.469012+00:00,1
5,13,"[{'id': 13, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 19786, 'text': 'Table 4-11: Effects and Mitigation Measures Raised at the Round 2...",{},2021-06-08 21:27:06.468972+00:00,2021-06-08 21:27:06.468979+00:00,1
6,12,"[{'id': 12, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 24525, 'text': 'Table 11-13 Site Specific Wildlife Mitigation Measures Recommende...",{},2021-06-08 21:27:06.468936+00:00,2021-06-08 21:27:06.468943+00:00,1
7,11,"[{'id': 11, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 351, 'text': 'Table 21-23 Cumulative Effects Initial Screening ï¿½ Social and Cul...",{},2021-06-08 21:27:06.468904+00:00,2021-06-08 21:27:06.468911+00:00,1
8,10,"[{'id': 10, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 15394, 'text': 'Table 2 TLRU Information and Mitigation ï¿½ Les Atikamekw de Mana...",{},2021-06-08 21:27:06.468870+00:00,2021-06-08 21:27:06.468877+00:00,1
9,9,"[{'id': 9, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 25965, 'text': 'Table 6-7 Agricultural Soils Ratings ï¿½ Infrastructure Physiogr...",{},2021-06-08 21:27:06.468830+00:00,2021-06-08 21:27:06.468837+00:00,1
